In [24]:
import geemap
import ee

In [25]:
# set our initial map parameters for lake 
center_lat = 40.1154
center_lon = -101.1108
zoomlevel = 6

In [26]:
# initialize our map
myMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
myMap

Map(center=[40.1154, -101.1108], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [27]:
aoi = ee.FeatureCollection(myMap.draw_features).geometry()

In [28]:
def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)
    
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2020-01-01","2020-12-31").filterBounds(aoi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5)).map(se2mask).median()

rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}

# initialize our map
map2 = geemap.Map()
map2.centerObject(aoi, 15)
map2.addLayer(se2.clip(aoi), rgbViz, "S2")

map2.addLayerControl()
map2

Map(center=[40.158024070306745, -101.11043436511362], controls=(WidgetControl(options=['position', 'transparen…

In [29]:
import os

In [30]:
out_dir = os.path.join(os.path.expanduser('~'), 'flodata')
filename = os.path.join(out_dir, 'swamson_lake_median.tif')

In [31]:
image = se2.clip(aoi)
geemap.ee_export_image(image, filename=filename, scale=10, region=aoi, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Hp PC\flodata
